In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers.initializers import variance_scaling_initializer

import skimage.io as io
import numpy as np
from matplotlib import pyplot as plt

import deepdish as dd
from os.path import join, dirname

import os.path as osp
import os
import sys
import pickle
curr_path = osp.dirname(os.getcwd())
model_dir = osp.join(curr_path, '..', 'models')

smpl_model_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\models\neutral_smpl_with_cocoplus_reg.pkl'
smpl_face_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\src\tf_smpl\smpl_faces.npy'

from src.batch_lbs import batch_rodrigues
from src.batch_smpl import SMPL
from src.projection import batch_orth_proj_idrot
from src.util import image as img_util

# !pip install import_ipynb
import import_ipynb
from _DemoTransformation import *

importing Jupyter notebook from _DemoTransformation.ipynb


In [2]:
with open(smpl_model_path, 'rb') as f:
    dd = pickle.load(f, encoding='latin1')

In [3]:
[k + ': ' + str(np.array(dd[k]).shape) for k in dd.keys()]

['J: (24, 3)',
 'weights_prior: (6890, 24)',
 'posedirs: (6890, 3, 207)',
 'bs_style: ()',
 'v_template: (6890, 3)',
 'cocoplus_regressor: ()',
 'J_regressor_prior: ()',
 'f: (13776, 3)',
 'J_regressor: ()',
 'kintree_table: (2, 24)',
 'weights: (6890, 24)',
 'bs_type: ()',
 'shapedirs: (6890, 3, 10)']

In [4]:
print(dd['cocoplus_regressor'].T.todense().shape)
print(dd['J_regressor_prior'].T.todense().shape)
print(dd['J_regressor'].T.todense().shape)

(6890, 19)
(6890, 24)
(6890, 24)


In [5]:
joint_type='cocoplus'
dtype=tf.float32

# Mean template vertices  [6890 x 3]
v_template = tf.constant(
                dd['v_template'],
                name='v_template',
                dtype=dtype)

# Size of mesh [Number of vertices (6890) x 3]
size = [v_template.shape[0].value, 3]

# Size of beta [10]
num_betas = dd['shapedirs'].shape[-1]

# Shape blend shape basis: [6980 x 3 x 10]
# reshaped to [6980*3 x 10], transposed to [10 x 6980*3]
shapedir = np.reshape(dd['shapedirs'], [-1, num_betas]).T
shapedirs = tf.constant(
                shapedir, 
                name='shapedirs', 
                dtype=dtype)

# Regressor for joint locations given shape - [6890 x 24]
J_regressor = tf.constant(
                dd['J_regressor'].T.todense(),
                name="J_regressor",
                dtype=dtype)

# Pose blend shape basis: [6890 x 3 x 207], reshaped to [6890*3 x 207], transposed to [207, 6890*3]
num_pose_basis = dd['posedirs'].shape[-1]
posedirs = np.reshape(dd['posedirs'], [-1, num_pose_basis]).T
posedirs = tf.constant(
                posedirs, 
                name='posedirs', 
                dtype=dtype)

# indices of parents for each joints
parents = dd['kintree_table'][0].astype(np.int32)

# LBS weights: [6890 x 24]
weights = tf.constant(
                dd['weights'],
                name='lbs_weights',
                dtype=dtype)

# This returns 19 keypoints: [6890 x 19]
joint_regressor = tf.constant(
                dd['cocoplus_regressor'].T.todense(),
                name="cocoplus_regressor",
                dtype=dtype)

if joint_type == 'lsp':  # 14 LSP joints!
    joint_regressor = joint_regressor[:, :14]

In [6]:
def load_mean_param():
    import deepdish as dd
    from os.path import join, dirname
    
    mean = np.zeros((1, 85))
    # Initialize scale at 0.9
    mean[0, 0] = 0.9
    mean_path = join(dirname(smpl_model_path), 'neutral_smpl_mean_params.h5')
    mean_vals = dd.io.load(mean_path)

    mean_pose = mean_vals['pose']
    # Ignore the global rotation.
    mean_pose[:3] = 0.
    mean_shape = mean_vals['shape']

    # This initializes the global pose to be up-right when projected
    mean_pose[0] = np.pi

    mean[0, 3:] = np.hstack((mean_pose, mean_shape))
    mean = tf.constant(mean, tf.float32)
    mean_var = tf.Variable(mean, name="mean_param", dtype=tf.float32, trainable=True)
    init_mean = tf.tile(mean_var, [1, 1])
    return init_mean

theta_prev = load_mean_param()

theta = theta_prev[:, 3:(3 + 72)]
beta = theta_prev[:, (3 + 72):]

get_skin = False
name = None

In [7]:
num_batch = tf.shape(beta)[0]

# 1. Add shape blend shapes
# [[N x 10] x [10 x 6890*3]] = [N x 6890*3], reshaped to [N x 6890 x 3]
v_shaped = tf.reshape(
                tf.matmul(beta, shapedirs, name='shape_bs'),
                [-1, size[0], size[1]]) + v_template


# 2. Infer shape-dependent joint locations. J is [N, 24, 3]
Jx = tf.matmul(v_shaped[:, :, 0], J_regressor)
Jy = tf.matmul(v_shaped[:, :, 1], J_regressor)
Jz = tf.matmul(v_shaped[:, :, 2], J_regressor)
J = tf.stack([Jx, Jy, Jz], axis=2)


# 3. Add pose blend shapes
# [N x 24 x 3 x 3]
Rs = tf.reshape(
            batch_rodrigues(
                tf.reshape(theta, [-1, 3])), 
            [-1, 24, 3, 3])

# Ignore global rotation. reshaped to [N, 207]
pose_feature = tf.reshape(Rs[:, 1:, :, :] - tf.eye(3), [-1, 207])

# [[N x 207] x [207, 20670]] -> [N x 6890 x 3]
v_posed = tf.reshape(
                tf.matmul(pose_feature, posedirs),
                [-1, size[0], size[1]]) + v_shaped


#4. Get the global joint location
J_transformed, A = batch_global_rigid_transformation(Rs, J, parents)


# 5. Do skinning:
# weights is [6890, 24], W is [N x 6890 x 24]
W = tf.reshape(
                tf.tile(weights, [num_batch, 1]), 
                [num_batch, -1, 24])

# [[N x 6890 x 24] x [N x 24 x 16]] -> [N x 6890 x 16]
# reshaped to [N x 6890 x 4 x 4]
T = tf.reshape(
                tf.matmul(W, tf.reshape(A, [num_batch, 24, 16])),
                [num_batch, -1, 4, 4])

# [N x 6890 x 3] -> [N x 6890 x 4] (homogeneous matrix)
v_posed_homo = tf.concat(
                [v_posed, tf.ones([num_batch, v_posed.shape[1], 1])], 2)

# [[N x 6890 x 4 x 4], [N x 6890 x 4 x 1]] -> [N x 6890 x 4 x 1]
v_homo = tf.matmul(T, tf.expand_dims(v_posed_homo, -1))

# [N x 6890 x 4 x 1] -> [N x 6890 x 3]
verts = v_homo[:, :, :3, 0]

# Get cocoplus or lsp joints:
joint_x = tf.matmul(verts[:, :, 0], joint_regressor)
joint_y = tf.matmul(verts[:, :, 1], joint_regressor)
joint_z = tf.matmul(verts[:, :, 2], joint_regressor)
joints = tf.stack([joint_x, joint_y, joint_z], axis=2)

# return verts, joints, Rs

W0311 17:46:29.353174 10060 deprecation.py:323] From <string>:39: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
